## Please provide your inputs as followed:

- **input_sdate**: The start date of your period of interest in the format **dd-mm-YYYY**
- **input_edate**: The end date of your period of interestin the format **dd-mm-YYYY**
- **geometry**: The region of interest. Please provide this in a **bounding box format** (e.g. [10, -5, 25, 20])

In [3]:
start_date = "2023 08 20"
end_date = "2023 08 25"
region = "Australia"

## Library imports 

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import netCDF4 as nc4
import xarray as xr
import fsspec
import numpy as np
import xarray as xr
import planetary_computer
import pystac_client
import geopandas as gpd
import pandas as pd
import cartopy.crs as ccrs
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime, time
import json
import stackstac

## Starting up PySTAC client

In [5]:
# Initialize PySTAC client for data query
planetary_computer.set_subscription_key("c27669c4bdec434d804e2bd738cb16fc")
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [ ]:
#The acceptable formats are dd mm YYYYY, YYYY mm dd, dd-mm-YYYY, YYYY-mm-dd, dd/mm/YYYY, YYYY/mm/dd
start_date = "2023 08 20"
end_date = "25 08 2023"
region = [174.563615, -36.893762, 174.860246, -36.717901] #Auckland NZ

## User Input

### Processing of user input

In [6]:
# Function to convert date format 
def convert_format_date(input_date):
    correct_formats = ["%d %m %Y", "%Y %m %d", "%d/%m/%Y", "%Y/%m/%d", "%d-%m-%Y", "%Y-%m-%d"]
    
    for format_str in correct_formats:
        try:
            date_obj = datetime.strptime(input_date, format_str)
            formatted_date = date_obj.strftime("%Y-%m-%d")
            return formatted_date
        except ValueError: # Raised if input format is not compatible with set standard 
            pass
    
    raise ValueError("Invalid data format")

# Convert user start date format
try:
    start_date = convert_format_date(start_date)
except ValueError:
    print("Invalid start date format. Please check the acceptable formats")
            
# Convert user end date format
try:
    end_date = convert_format_date(end_date)
except ValueError:
    print("Invalid end date format. Please check the acceptable formats")

date_period = start_date + "/" + end_date 
print(date_period)

2023-08-20/2023-08-25


## Search for product

In [7]:
# Get geopandas in-built naturalearth_lowres dataset
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

/tmp/ipykernel_391/755612632.py:2: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


In [7]:
# Get geopandas in-built naturalearth_lowres dataset
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

/tmp/ipykernel_391/755612632.py:2: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


In [8]:
def search_catalog(region, date_period):
    search_params = {
        "collections": "sentinel-5p-l2-netcdf",
        "datetime": date_period,
        "query": {"s5p:processing_mode": {"eq": "OFFL"}, "s5p:product_name": {"eq": "ch4"}},
    }

    if isinstance(region, list) and len(region) == 4:
        # bbox input is a list of four coordinates [west, south, east, north]
        search_params["bbox"] = region
    else:
        # country input
        # Extract coordinates of specified country and load into a JSON object
        ROI = world[world["name"] == region]
        gjson = json.loads(ROI.to_json())
        # Create a MultiPolygon GeoJSON structure
        coordinates = gjson["features"][0]["geometry"]["coordinates"]
        if gjson["features"][0]["geometry"]["type"] == "Polygon": 
            coordinates = [coordinates]  
            # Convert to MultiPolygon
        search_params["intersects"] = {
            "type": "MultiPolygon", 
            "coordinates": coordinates,
        }

    search = catalog.search(**search_params)
    item = search.item_collection()

    return item

# Use search_catalog function with a single variable "Image" for both bbox and country
result = search_catalog(region=region, date_period=date_period)

# Print the result
print(f"Number of items found: {len(result)}")

Number of items found: 17


In [9]:
result[-1]

id: S5P_L2_CH4____20230820T030443_20230820T044612_30315
"bbox: [-180.0, -87.81228, 180.0, 90.0]"
"s5p:ch4: {'input_band': ['L1B_RA_BD7', 'L1B_RA_BD8', 'L1B_RA_BD6', 'L2__CO____', 'L2__FRESCO', 'L2__NP_BD6', 'L2__NP_BD7'], 'irradiance_accompanied': ['L1B_IR_SIR', 'L1B_IR_UVN'], 'geolocation_grid_from_band': 7}"
datetime: 2023-08-20T03:55:29Z
platform: Sentinel 5 Precursor
"s5p:shape: [4173, 215]"
instruments: ['TROPOMI']
end_datetime: 2023-08-20T04:24:41Z
constellation: Sentinel-5P
start_datetime: 2023-08-20T03:26:17Z
s5p:product_name: ch4


In [10]:
print([item.properties["s5p:spatial_resolution"] for item in result])

[[5500, 7000], [5500, 7000], [5500, 7000], [5500, 7000], [5500, 7000], [5500, 7000], [5500, 7000], [5500, 7000], [5500, 7000], [5500, 7000], [5500, 7000], [5500, 7000], [5500, 7000], [5500, 7000], [5500, 7000], [5500, 7000], [5500, 7000]]


In [11]:
data = (
    stackstac.stack(
        result,
        epsg=4326,
        assets=["ch4"],
        resolution= (5000,7000)
    )
)

data

/srv/conda/envs/notebook/lib/python3.11/site-packages/stackstac/prepare.py:363: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


<xarray.DataArray 'stackstac-e9a201ef33ea144e3a3b4d11be05c940' (time: 17,
                                                                band: 1, y: 2,
                                                                x: 2)>
dask.array<fetch_raster_window, shape=(17, 1, 2, 2), dtype=float64, chunksize=(1, 1, 2, 2), chunktype=numpy.ndarray>
Coordinates: (12/20)
  * time                                   (time) datetime64[ns] 2023-08-20T0...
    id                                     (time) <U51 'S5P_L2_CH4____2023082...
  * band                                   (band) <U3 'ch4'
  * x                                      (x) float64 -5e+03 0.0
  * y                                      (y) float64 7e+03 0.0
    s5p:spatial_resolution                 object {7000, 5500}
    ...                                     ...
    platform                               <U20 'Sentinel 5 Precursor'
    sat:absolute_orbit                     (time) int64 30315 30316 ... 30388
    s5p:processing_mode                    <U4 'OFFL'
    s5p:product_type                       <U10 'L2__CH4___'
    title                                  <U20 'Methane Total Column'
    epsg                                   int64 4326
Attributes:
    spec:           RasterSpec(epsg=4326, bounds=(-5000, -7000, 5000, 7000), ...
    crs:            epsg:4326
    transform:      | 5000.00, 0.00,-5000.00|\n| 0.00,-7000.00, 7000.00|\n| 0...
    resolution_xy:  (5000, 7000)

In [23]:
print([value for value in data.x.values])

[-5000.0, 0.0]


In [24]:
data = (
    stackstac.stack(
        result,
        epsg=4087,
        assets=["ch4"],
        resolution= (5000,7000)
    )
)

data

/srv/conda/envs/notebook/lib/python3.11/site-packages/stackstac/prepare.py:363: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


<xarray.DataArray 'stackstac-d79114a60c9ec570fbc85c73eb5f0daa' (time: 4029,
                                                                band: 1,
                                                                y: 2864, x: 8016)>
dask.array<fetch_raster_window, shape=(4029, 1, 2864, 8016), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/19)
  * time                                   (time) datetime64[ns] 2019-08-06T0...
    id                                     (time) <U51 'S5P_L2_CH4____2019080...
  * band                                   (band) <U3 'ch4'
  * x                                      (x) float64 -2.004e+07 ... 2.004e+07
  * y                                      (y) float64 1.002e+07 ... -1.002e+07
    sat:platform_international_designator  <U9 '2017-064A'
    ...                                     ...
    s5p:product_type                       <U10 'L2__CH4___'
    s5p:collection_identifier              (time) <U2 '01' '01' ... '03' '03'
    s5p:ch4                                object {'input_band': ['L1B_RA_BD7...
    s5p:processing_mode                    <U4 'OFFL'
    title                                  <U20 'Methane Total Column'
    epsg                                   int64 4087
Attributes:
    spec:           RasterSpec(epsg=4087, bounds=(-20040000, -10024000, 20040...
    crs:            epsg:4087
    transform:      | 5000.00, 0.00,-20040000.00|\n| 0.00,-7000.00, 10024000....
    resolution_xy:  (5000, 7000)

## Process queried data into a dataset

In [9]:
item_links = [item.assets['ch4'].href for item in result]
item_links

f = fsspec.open_files(item_links)
f = [file.open() for file in f]

In [ ]:
datasets = [xr.open_dataset(nc_file, group="PRODUCT", engine="h5netcdf") for nc_file in f]

In [ ]:
# Initialize the map
fig, ax = plt.subplots(1, 1, figsize=(12, 8), subplot_kw={'projection': ccrs.PlateCarree()})
continent_borders = world.dissolve(by='continent')
continent_borders.boundary.plot(ax=ax, linewidth=1, color='black')

for item in f: # For each opened item in query
    ds = xr.open_dataset(item, group="PRODUCT", engine="h5netcdf") # Create a dataset
    
    for time in range(ds.dims["time"]): # For each time within a dataset
        
        # Extract the relevant data (assuming the variable name is 'methane_mixing_ratio_bias_corrected')
        data = ds['methane_mixing_ratio_bias_corrected'][0, :, :] # 
        #print(data.values)
        lon = ds['longitude'].values.squeeze()
        lat = ds['latitude'].values.squeeze()

        # Calculate vmin and vmax for color normalization
        vmin, vmax = np.nanpercentile(data, [1, 99])

        # Plot the data
        norm = matplotlib.colors.Normalize(vmin=vmin, vmax=vmax)
        colormesh = ax.pcolormesh(lon, lat, data.values, cmap="Spectral", norm=norm, transform=ccrs.PlateCarree(), alpha=0.9, rasterized=True)
        
fig.colorbar(colormesh, pad=0.05, shrink=0.35, label="methane (mol/m2)")

ax.set_xlim(bbox[0], bbox[2])
ax.set_ylim(bbox[1], bbox[3])

plt.show()

## Plot Time Series